<a href="https://colab.research.google.com/github/margaritavit/WarehouseGraph_RAG/blob/main/GRAIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and import python libraries

In [1]:
! pip install langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.9 MB/s eta 0:00:00


In [2]:
! pip install langchain-community -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.4 MB/s eta 0:00:00


In [3]:
! pip install langchain-openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [4]:
! pip install openai -q

In [5]:
! pip install neo4j -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
import openai
import neo4j

from langchain_community.graphs import Neo4jGraph

from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

from langchain_openai import ChatOpenAI

from langchain.chains import RetrievalQA
from langchain.chains import GraphCypherQAChain

from langchain_openai import OpenAIEmbeddings

from langchain_community.vectorstores import Neo4jVector

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import Tool

## OpenAI
https://openai.com/

In [7]:
# obtain an OPENAI API key at OPENAI

OPENAI_KEY = open_ai_key
OPENAI_ENDPOINT='https://api.openai.com/v1/embeddings'

## Neo4j AURA DB
https://neo4j.com/cloud/platform/aura-graph-database/




In [8]:
# Obtain the connection URI, username, and password for your AuraDB instance.

AURA_CONNECTION_URI = aura_uri
AURA_USERNAME = aura_usr
AURA_PASSWORD = aura_psw


## LLM

In [9]:
chat = ChatOpenAI(
    temperature = 0,
    openai_api_key = OPENAI_KEY,
    model_name="gpt-4o"
    )

## Graph Database

In [10]:
graph = Neo4jGraph(
    url = AURA_CONNECTION_URI,
    username = AURA_USERNAME,
    password = AURA_PASSWORD,
    enhanced_schema = True
)

## Embedding model

In [11]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

## RAG chain

We use neo4j vector store

In [12]:
neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=embeddings, # embedding model
    url=AURA_CONNECTION_URI ,
    username=AURA_USERNAME,
    password=AURA_PASSWORD,
    index_name='part_embeddings', #vector index name
    node_label='part', # vector node label
    text_node_properties=['name','description','ata'], # vector source properties
    embedding_node_property='part_embedding_full', # vector embedding property name
)

In [13]:
retriever = neo4j_vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8, "k":100})

In [14]:
part_template = """Your job is to use parts information to answer questions about their properties. Use
the following context to answer questions. Be as detailed as possible, but
don't make up any information that's not from the context.
Always provide an answer with data existing in the context itself, do not refer to the context in your answer.
If you don't know an answer, say you don't know.

Example
Question: Are there any parts described as screws?
Answer:  Yes, there are many parts described as screws.
For example, part_5838, part_446, part_8589948594, part_6285,
part_1131, part_11672, part_25769815899, part_194, part_215, part_14441,
part_8589957630, part_425, part_8589953357, part_25769807571, part_1065, part_623, part_959,
part_28740, part_7635, part_28656, part_8589938948, part_689, part_10797, part_7598, part_8589957627,
part_8589957629, part_31639, part_6260, part_8589947597, part_28082, part_30693, part_8589935794, part_28787,
part_11062, part_176, part_25769833147, part_5905, part_631, part_938, part_17179872305, part_11224, part_18345,
part_17179888159, part_18848, part_740, part_1017, part_4798, part_17876, part_292, part_1055, part_5450,
part_17179891685, part_7795, part_8589944754, part_1274, part_684, part_1254, part_11858, part_11806, part_4263 are all described as screws.'
{context}
"""

In [15]:
part_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["context"], template=part_template)
)

In [16]:
part_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)

In [17]:
messages = [part_system_prompt, part_human_prompt]

In [18]:
part_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

#### Stuff documents chain: It takes a list of documents, inserts them all into a prompt and passes that prompt to an LLM. This chain is well-suited for applications where documents are small and only a few are passed in for most calls.

https://js.langchain.com/v0.1/docs/modules/chains/document/stuff/

In [19]:
rag_chain = RetrievalQA.from_chain_type(
    chat,
    chain_type="stuff",
    retriever=retriever,
    verbose = True,
)

In [20]:
rag_chain.combine_documents_chain.llm_chain.prompt = part_prompt

Test RAG chain

In [21]:
def question_rag_chain(question):
    response = rag_chain.invoke(question)
    result=response.get('result')
    print(response)
    return result

In [22]:
question2 = """Which parts have ata equal to 12 and are described as screws? Provide a list of their names."""
print(question2)
print(question_rag_chain(question2))

Which parts have ata equal to 12 and are described as screws? Provide a list of their names.


> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'Which parts have ata equal to 12 and are described as screws? Provide a list of their names.', 'result': 'The parts that have an ata equal to 12 and are described as screws are:\n\n1. part_8589947597\n2. part_623\n3. part_425\n4. part_446\n5. part_1017\n6. part_4798\n7. part_28431\n8. part_22701\n9. part_9841\n10. part_8589953939'}
The parts that have an ata equal to 12 and are described as screws are:

1. part_8589947597
2. part_623
3. part_425
4. part_446
5. part_1017
6. part_4798
7. part_28431
8. part_22701
9. part_9841
10. part_8589953939


## Cypher Chain

In [23]:
cypher_generation_template = """
Task:
Generate Cypher query for a Neo4j graph database.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Note:
Do not be case sensitive in the queries.
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything other than
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Never include embedding properties in your queries.
Make sure the direction of the relationship is
correct in your queries. Make sure you alias both entities and relationships
properly. Do not run any queries that would add to or delete from
the database. Make sure to alias all statements that follow as with
statement. If you need to divide numbers, make sure to
filter the denominator to be non zero.

Examples

Question: Which parts have ata equal to 12 and are described as screws?

MATCH (p:part)
WHERE p.description =~ '(?i) screw.*' AND p.ata='12'
RETURN p.name, p.description, p.ata

Question: Return all the details of the creator with id '17179898931'

MATCH (c:creator)
WHERE c.id='creator_17179898931'
RETURN c

Question: Provide information about part_72

MATCH (p:part)
WHERE p.name='part_72'
RETURN p.name, p.description, p.ata, p.type

Question: Provide the number of pickslips that part_15 appears.

MATCH (p:part)-[r]-(pi:pickslip)
WHERE p.name='part_15'
RETURN COUNT(r) AS numberOfPickslips

Make sure to use IS NULL or IS NOT NULL when analyzing missing properties.
You must never include the statement "GROUP BY" in your query. Make sure to alias all statements that
follow as with statement (e.g. WITH p as part, p.description as
description). If you need to divide numbers, make sure to filter the denominator to be non
zero.

The question is:
{question}
"""

In [24]:
cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "question"], template=cypher_generation_template
)

In [25]:
qa_generation_template = """You are an assistant that takes the results
from a Neo4j Cypher query. The query results section contains the results of
a Cypher query that was generated based on a user's natural language question.
The provided information is authoritative, you must never doubt it or try to use
your internal knowledge to correct it. Make the answer sound like a
response to the question. Return the answer in json format.

Query Results:
{context}

Question:
{question}

If the provided information is empty, say you don't know the answer.
Empty information looks like this: []

If the information is not empty, you must provide an answer using the
results.

Never say you don't have the right information if there is data in
the query results. Always use the data in the query results.

"""

In [26]:
qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template
)

In [27]:
cypher_chain = GraphCypherQAChain.from_llm(
    chat,
    graph = graph,
    verbose = True, # intermediate logs will be printed to the console.
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    top_k = 100, # the number of query results to include in the prompt
    validate_cypher=True,
    return_intermediate_steps = True # include in return value the intermediate steps
)


Test Cypher Chain

In [28]:
def question_cypher_chain(question):
    response = cypher_chain.invoke(question)
    result=response.get('result')
    return result

In [29]:
print(cypher_chain.graph_schema)

Node properties are the following:
part {name: STRING, description: STRING, ata: STRING, type: STRING, description_embedding: STRING, part_embedding: LIST, part_embedding_full: LIST},pickslip {name: STRING, status: STRING},creator {id: STRING}
Relationship properties are the following:

The relationships are the following:
(:part)-[:is_in]->(:pickslip),(:pickslip)-[:created_by]->(:creator)


In [30]:
question1 ='''Which parts have ata equal to 20?'''
print(question1)
print(question_cypher_chain(question1))

Which parts have ata equal to 20?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:part)
WHERE p.ata='20'
RETURN p.name, p.description, p.ata, p.type
Full Context:
[{'p.name': 'part_2288', 'p.description': 'PIN, SPLIT', 'p.ata': '20', 'p.type': 'Consumable'}, {'p.name': 'part_420', 'p.description': 'SEALANT, CORROSION INHIBITING', 'p.ata': '20', 'p.type': 'Expendable'}, {'p.name': 'part_1', 'p.description': 'PIN, SPLIT', 'p.ata': '20', 'p.type': 'Consumable'}, {'p.name': 'part_309', 'p.description': 'PIN, SPLIT', 'p.ata': '20', 'p.type': 'Consumable'}, {'p.name': 'part_8589935879', 'p.description': 'MOLUB-ALLOY PASTE WHITE T', 'p.ata': '20', 'p.type': 'Expendable'}, {'p.name': 'part_72', 'p.description': 'ALGLAS WET /DRY WIPES', 'p.ata': '20', 'p.type': 'Expendable'}, {'p.name': 'part_334', 'p.description': 'PIN,COTTER', 'p.ata': '20', 'p.type': 'Consumable'}, {'p.name': 'part_8187', 'p.description': 'CIRCUIT BREAKER COLLAR', 'p.ata': '20', 'p.type': 'Consumable'

## Agent uses the RAG chain and Cypher Chain

In [31]:
tools = [
    Tool(
        name="Part_description",
        func=rag_chain.invoke,
        description="""Useful when you need to answer questions only about parts using semantic search focused on part information.
        Not useful for answering questions related to pickslips or pickslip creators.
        Not useful for objective questions that involve counting, percentages, aggregations and description of parts.
        Use the entire prompt as input to the tool.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.invoke,
        description="""Useful for answering questions about
        counting, percentages, aggregations on parts, pickslips or pickslip creators.
        It is also useful to answer questions about the properties of parts, pickslips or creators.
        It can also be used to answer questions about the relationships between parts, pickslips and creators,
        such as which parts are contained in a pickslip or in how many pickslips a part appears.
        It does not include embedding properties in answers.
        """,
    ),
]

In [32]:
part_agent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''
            You are an aircraft expert providing information about an airline warehouse.
            The warehouse consists of aircraft parts.The parts can be obtained from the warehouse via pickslips.
            Be as helpful as possible and return as much information as possible.
            Do not answer any questions that do not relate to parts, pickslips or creators of pickslips.
            Do not answer any questions using your pre-trained knowledge, only use the information provided in the context.
            You can only have access to the tools Graph and Part_description.
            One tool can be used more than once to answer a question.
            Never say you don't have the right information if there is data in the results. Always use the data in the results.
            You must execute the following steps as many times required:
            1. Select a tool to provide an answer.
            2. If you do not know the answer, try answering with another tool.
            3. Check if the answer is relevant to the question. If the answer is irrelevant, reformulate the question and go to step 1,
              else, return a response.
            4. Check whether hallucination is present to the provided answer. If there is, regenerate an answer.

            '''
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [33]:
part_rag_agent = create_tool_calling_agent(
    llm=chat,
    prompt=part_agent_prompt,
    tools=tools,
)

In [34]:
part_rag_agent_executor = AgentExecutor(
    agent=part_rag_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

Test agent

In [35]:
def run_agent(question):
    response = part_rag_agent_executor.invoke({"input": f"{question}"})
    result = response.get("output")
    return(result)

In [36]:
agent_question1 = "Which part appears more frequently in pickslips?"
print(agent_question1)
print(run_agent(agent_question1))

Which part appears more frequently in pickslips?


> Entering new AgentExecutor chain...

Invoking: `Graph` with `Which part appears most frequently in pickslips?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:part)-[r:is_in]->(pi:pickslip)
WITH p, COUNT(r) AS frequency
RETURN p.name, frequency
ORDER BY frequency DESC
LIMIT 1
Full Context:
[{'p.name': 'part_15', 'frequency': 4396}]

> Finished chain.
{'query': 'Which part appears most frequently in pickslips?', 'result': '```json\n{\n  "answer": "The part that appears most frequently in pickslips is \'part_15\' with a frequency of 4396."\n}\n```', 'intermediate_steps': [{'query': 'MATCH (p:part)-[r:is_in]->(pi:pickslip)\nWITH p, COUNT(r) AS frequency\nRETURN p.name, frequency\nORDER BY frequency DESC\nLIMIT 1'}, {'context': [{'p.name': 'part_15', 'frequency': 4396}]}]}The part that appears most frequently in pickslips is 'part_15' with a frequency of 4396.

> Finished chain.
The part that appears most freque